In [ ]:
import torch
import transformers

from groq import Groq
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

In [ ]:
# Initialize Groq client
client = Groq(api_key='')


In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model_whisper = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model_whisper.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe_whisper = pipeline(
    "automatic-speech-recognition",
    model=model_whisper,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

In [ ]:
openai_model_3="llama-3.1-70b-versatile"
openai_model = openai_model_3
def pipe_gpt(messages, temperature=0.67, model=openai_model_3):
    chat_completion = client.chat.completions.create(
        messages = messages,
    
        # The language model which will generate the completion.
        model=openai_model_3,
    
        #
        # Optional parameters
        #
    
        # Controls randomness: lowering results in less random completions.
        # As the temperature approaches zero, the model will become deterministic
        # and repetitive.
        temperature=0.7,
    
        # The maximum number of tokens to generate. Requests can use up to
        # 32,768 tokens shared between prompt and completion.
        max_tokens=4096,
    
        # Controls diversity via nucleus sampling: 0.5 means half of all
        # likelihood-weighted options are considered.
        top_p=1,
    
        # A stop sequence is a predefined or user-specified text string that
        # signals an AI to stop generating content, ensuring its responses
        # remain focused and concise. Examples include punctuation marks and
        # markers like "[end]".
        stop=None,
    
        # If set, partial message deltas will be sent.
        stream=False,
    )
    return chat_completion.choices[0].message.content

In [ ]:
structure_tags = '''We can influence the song structure with metatags, although the ai tends to have a mind of its own and follow it’s own pattern.

For a musical introduction.
[Intrumental Break]
[Short Instrumental Intro]

A hook is a repetitive phrase or instrumental. Try repeating a short line 2 – 4 times with or without the label.
[Hook]
[Catchy Hook]

A break is a few bars of the song where the lead instruments or singer go silent, and the accompanying instruments play. A [Break] can sometimes be used strategically to interrupt the current pattern.
[Break]
[Percussion Break]

Interlude is a useful tag to create an instrumental section within the lyrics.
[Interlude]
[melodic interlude]

An Outro can help to prime the song to end, and may create a loop to fade out in post edit.
[Outro]

Refrain seems to get more ‘creative’ when wrapping up the end of the song, while Big Finish may change the melody or tempo to create a climax.
[Refrain]
[Big Finish]

An end tag in the lyrics may work best alone as its own clip. Clear the Style Prompt, or add ‘End’ to the style description.  Use any of the following:
[End], [Fade Out], or [Fade to End].  Choose the one for the desired effect.'''

verse_chorus_tags = '''A pattern that works well for Continued songs is to have a verse and chorus in each clip. The tags are not necessary, the AI will build a song pattern from the lyrics with or without them.

Verses are usually rhythmic and restrained, while the chorus has more melody and energy. A chorus is usually the ‘hook’ of the song, when it repeats it makes the song feel intentional and emotional. 
[Verse]
[Chorus]

Add descriptive Style Words to metatags to guide how the lyrics should be sung.
[Sad Verse]
[Happy Chorus]

Use musical terms to influence the genre.
[Rapped Verse]
[Powerpop Chorus]

Lyrics are Stronger than Metatags
Metatags can ‘nudge’ the AI within the lyrics, but the lyric-structure, the current song pattern, and the Style Prompt, are stronger influences than the tags.

A fast rap song needs more words per line than a slow ballad.
Verse and Chorus need different syllable-per-line counts and phrasing, or they will sound the same and blur together.
It’s possible to have a song that’s just verses, when all the lyrics follow the same pattern and rhyme scheme.'''

prechorus_bridge_tags = '''Pre-chorus and bridge are for stray lyrics outside the main pattern. They build anticipation as the song transitions, and often don’t rhyme or appear to fit the meter.

[Pre-Chorus]
[Bridge]
AI-generated lyrics sometimes include a pre-chorus without labeling it, causing Chirp to sing it as an awkward extra line that doesn’t fit the meter.

Adding a metatag label tells Chirp this break in the song pattern is intentional, and it should be sung as its own pattern.

Lyrics generated by AI
[Verse]
Cruisin’ down the streets with nowhere to go
Miles of cars, it’s a never-ending show
Round and round, it’s like a crazy maze
Every parking space, a mirage, a haze
Anxiety’s building, it’s drivin’ me insane

[Chorus]
Driving in circles, looking for a spot
I’m runnin’ out of gas, I’m losing all my shots

AI Lyrics Edited to add a [Pre-chorus]
[Verse]
Cruisin’ down the streets with nowhere to go
Miles of cars, it’s a never-ending show
Round and round, it’s like a crazy maze
Every parking space, a mirage, a haze

[Pre-chorus]
Anxiety’s building, it’s drivin’ me insane

[Chorus]
Driving in circles, looking for a spot
I’m runnin’ out of gas, I’m losing all my shots

Pre-chorus is a lead-in to a chorus. A bridge can go anywhere. It may be enough to simply set apart the lyrics with any descriptive tag:
[Shout]
[Whimsical]
[Melancholy]'''

instrumental_tags = '''Songs can have instrumental sections which can be prompted the same as [Verse] and [Chorus], but without lyrics the landmarks aren’t as clear.

An instrumental ‘break’ can replace a verse standing as its own section, or might be a short bridge in the music. These seem to work best when only one is used at a time, but adding commas inside the prompt may work. Experiment!

Prompt examples include:
[Break]
[Instrumental Interlude]
[Melodic Bass]
[Percussion Break]
[Syncopated Bass]
[Fingerstyle Guitar Solo]
[Build]
[Bass Drop]

Stay in the genre
The genre is important! You may need to describe the instrument within the Style Prompt if you want to manipulate it with metatags.
A [Bass Drop] is a common feature of EDM-genre, but it makes no sense in an acoustic guitar solo.
A [Bluegrass Banjo Interlude] will be easier to conjure within a Country-genre song, but might not work at all within an Orchestral Symphony….
Then again, it might work if a ‘banjo’ is added to the style prompt.

Experiment with ‘instrumental lyrics’
Suno will sometimes respond to un-singable text as a musical instrument. a few lines of punctuation-only might help to force a short instrumental solo.

Less reliable, and sometimes hilarious, try onomatopoeic words that mimic the sounds of the musical instruments. Often they are sung as lyrics, but sometimes trigger the intended instrument.
[Percussion Break]
. .! .. .!
!! ... ! ! !

[sad trombone]
waah-Waaah-WAAH

[chugging guitar]
chuka-chuka-chuka-chuka'''

voice_tags = '''Style vocals can be used as song-section metatags
[Female Narrator]
[Diva Solo]
[Gospel Choir]
[Primal Scream]
[Rap Verse]
'''

example_song = '''style: duet,  male and female duet,  catchy,  futuristic,  electronic,  melodic,  melancholic,  upbeat,  dance-pop,  beat,  trap,  rap

lyrics: [Verse 1]
[Female Voice]
Morning light, it calls my name  
Whispers sweet, a lover’s game  
In the quiet, I hear the sound  
Heartbeat racing, pulling me down

[Pre-Chorus]
[Female Voice]
With every sip, a gentle kiss  
In the shadows, I find my bliss  
A fleeting touch, I can't resist  
In the shadows, I find my bliss

[Chorus]
[Female Voice]
Dancing with the stars, under neon skies  
Chasing dreams that never die  
In the night, I come alive  
In the night, I come alive

[Verse 2]
[Male Voice]
Eyes wide open, chasing the dawn  
In this world, I’m never gone  
A siren's call, I can't ignore  
Keeps me wanting, yearning for more

[Pre-Chorus]
[Male Voice]
With every sip, a gentle kiss  
In the shadows, I find my bliss  
A fleeting touch, I can't resist  
In the shadows, I find my bliss

[Chorus]
[Male Voice]
Dancing with the stars, under neon skies  
Chasing dreams that never die  
In the night, I come alive  
In the night, I come alive

[Bridge]
[Female Voice]
In the silence of the night, I hear the call  
A secret dance, a silent thrall  
Through the highs and the lows, I’ll stand tall 
In this embrace, I find it all

[Pre-Chorus]
[Female Voice]
With every sip, a gentle kiss  
In the shadows, I find my bliss  
A fleeting touch, I can't resist  
In the shadows, I find my bliss

[Chorus]
[Female Vocalist and Male Vocalist]
Dancing with the stars, under neon skies  
Chasing dreams that never die  
In the night, I come alive  
In the night, I come alive

[Outro]
[Male Vocalist]
Morning light, it calls my name  
In this dance, we are the same  
In the quiet, I hear the sound  
Heartbeat racing, pulling me down
'''

# Create ideas for Album of songs

In [ ]:
messages = [
    {"role": "system", "content": "You are an expert assistant that provides help with a genuis level vocabularly and creative mind."},
    {"role": "user", "content": "Create a list of songs as a storyboard for an album where the songs resemble the plot of a creative, imaginative, and engaging science fiction, fantasy novel with magic and assasins.  Create character names and personas instead of generic references like protagonist.  The genre for each song should be unique and resemble the character's personality (rap, rock, metal, etc.) Include rap as one of the genres. The list being with the album title and have the format [song_name] : [plot description] : [genre]"},
]

story_board = pipe_gpt(messages, 0.8, openai_model_3)
print(story_board)

# Create Song from an Idea

In [ ]:
album = "Shadows of Enchantment"
song = "Enchanted Reverie: In the realm of dreams, the dreamweaver, Lyra, guides lost souls through the ethereal landscape of the subconscious, offering solace and hope. Genre: New Age"
theme = f"Album: {album} - song: {song}."
goal = "The goal is to create a poetic song that transports the listener into a fantasy novel with a rich plot.  Use a genius level vocabularly, like Emily Dickenson. This song is like a chapter in a novel.  Use rhymes and cultural lingo to make the song engaging and emotional using creative and imaginative plots described by a poetic song.  Avoid generalization in favor of emotional details (joy, pain, anger, happiness, love, hate, etc.). Songs about battles should have details, like how beings are killed or survive in an emotional way.  Avoid general references, such as to deepest fears, rather be specific about the fear and how it was overcome.  Create emotional songs with rich plots."

messages = [
    {"role": "system", "content": "You are an expert assistant with a genius level vocabularly and creative mind and you are a songwriter who writes amazing hit songs that are number one in the charts.  You are poetic and use rhymes, a world renowned song writer and intellectual, the world's smartest and best.  Your songs have meta tags for structure, verse and chorus, lyrics outside the main patteres, instrumental sections, and singing voice."},
    {"role": "user", "content": f"Compose a song and lyrics.  For the song composition, you use any of the following meta tags. Set the song structure with strucute_tags:{structure_tags}. Set the verse and chorus with verse_chorus_tags: {verse_chorus_tags}. For lyrics outside the main pattern, use prechorus_bridge_tags: {prechorus_bridge_tags}. Set instrumental sections with instrumental_tags: {instrumental_tags}. Influce the singing voice with voice_tags: {voice_tags}."},
    {"role": "user", "content": f"Use plenty of meta tags to make a hit song, it's not just the lyrics, but the music composition too."},
    {"role": "user", "content": f"Create a title and lyrics for a hit song about {theme}  {goal}  Here is the storyboard: {story_board}"},
]

lyrics = pipe_gpt(messages, 0.7, openai_model_3)

messages.append({"role": "assistant", "content": lyrics})
messages.append({"role": "user", "content": "act as an expert music critic and suggest changes to improve the song lyrics to be a number one hit.  You favor rich, emotional details over generalizations.  For example, a map to the constellations doesn't say which constellation.  A map of the stars doesn't say which stars.  A reference to a battle doesn't say how the battle is fought (armor, weapons, land features, etc.)"})

criticism = pipe_gpt(messages, 0.7, openai_model)

messages.append({"role": "assistant", "content": criticism})
messages.append({"role": "user", "content": "Act as an expert songwriter that writes amazing hit songs that don't have misinformation. Revise the title to be short and sweet, with revised lyrics to address the criticisms."})

revised_lyrics = pipe_gpt(messages, 0.7, openai_model_3)
print(revised_lyrics)

In [ ]:
messages = [
    {"role": "system", "content": "You are an expert assistant and music producer that creates number one hit songs."},
    {"role": "user", "content": f"Describe a music genre and instruments to make a hit song about {theme} with these lyrics: {revised_lyrics}"},
]

genre = pipe_gpt(messages, 0.5, openai_model_3)
print(genre)

# Create a new song using another song as inspiration

In [ ]:
lyrics = '''[Verse]
Gears are turning notes are churning
In a corporate dance
Legal papers maps and capers
All in sync by chance

[Verse 2]
Data flowing systems glowing
In a rhythm true
Answers finding documents binding
All for me and you

[Chorus]
Retrieval augmented
That's how they keep us sorted
In the world of legal orders
CLOC’s the way we’re boarded

[Verse 3]
Queries fetching records catching
In a swift ballet
Files aligning smoothly signing
Every single day

[Bridge]
Agile program tick tock tick tock
Moving with precision
From the past and information cast
Shows us clear vision

[Chorus]
Retrieval augmented
That's how they keep us sorted
In the world of legal orders
CLOC’s the way we’re boarded'''

genre = 'ragtime'
messages = [
    {"role": "system", "content": "You are an expert assistant and music producer that creates number one hit songs and provides help.  Your songs have meta tags for structure, verse and chorus, lyrics outside the main patteres, instrumental sections, and singing voice."},
    {"role": "user", "content": f"Compose a song and lyrics.  For the song composition you use any of the following metatags. Set the song structure with strucute_tags:{structure_tags}. Set the verse and chorus with verse_chorus_tags: {verse_chorus_tags}. For lyrics outside the main pattern, use prechorus_bridge_tags: {prechorus_bridge_tags}. Set instrumental sections with instrumental_tags: {instrumental_tags}. Influce the singing voice with voice_tags: {voice_tags}."},
    {"role": "user", "content": f"Add and use more structure metatags to make a hit song, it's not just the lyrics, but the music composition too."},
    {"role": "user", "content": f"We want to modify these lyrics: {lyrics}, but keep the theme of RAG (retrieval augmented generation) and how it's used in corporate legal operations (CLOC)"},
    {"role": "user", "content": f"The goal is to create a hit song in a genre described as: {genre}.  Create a title and revised lyrics.  If there are no lyrics, create an idea for a song and write the lyrics.  Don't use the words: neon, echoes, or whisper."},
]

revised_lyrics = pipe_gpt(messages, 0.67)
print(revised_lyrics)

In [ ]:
messages = [
    {"role": "system", "content": "You are an expert assistant and music producer that creates number one hit songs.  Your songs have meta tags for structure, verse and chorus, lyrics outside the main patteres, instrumental sections, and singing voice."},
    {"role": "user", "content": f"Describe a music genre and instruments to make a hit song with these lyrics: {revised_lyrics}"},
]

genre = pipe_gpt(messages, 0.5)
print(genre)

## Create song from event or facts

In [ ]:
facts = '''
VACCINATION

Speaking about this particular topic could be very controversial and most of the medical professionals might even say that we are doing a disservice to humanity, because somebody could die due to the bite of a certain tropical insect, for instance, however, we must stress that undoubtedly there are very good vaccines, which have saved human lives, the problem that we have now has to do with what’s being added to these vaccines.

If we think that a vaccine is a good product and the government wants to inoculate many people, and a separate organization with an evil agenda that have a lot of power come along and say that they want to put some additive to these vaccines, then the vaccines are altered. Then, when a doctor gives an injection to a patient he/she believes that the patient is receiving something good, it's important to say that the vast majority of the doctors are actually good people, who are concerned and care about their patients, but they are kept in ignorance with regard to the content of the vaccines.
 
Then, in our estimation there are some inoculations which are not good and certainly many children will grow up with them, so, we suggest that those parents look at this issue and pay attention that in America a law has been passed, which is not yet in effect and you will not be able to send your child to kindergarten unless he/she has received all the injections, the main argument they have is that they don’t want to spread a disease to all the rest of the children, but also they will know when the vaccine was inoculated to a population based on the age range, so the risk is big and the cost of being against recent vaccines very present.

Then, personally, at this precise moment, I would not receive any vaccine offered to me at all, not until the system changes, I want to be positive on this issue, but until these evil people are removed from their positions of power, the medical system cleared out and we are informed on what exactly these injections have, what they have put in the food and drinking water, I cannot accept it.  

At this moment ordinary people do not know what they are eating, what they are drinking or what’s being  injected into them because they are not being told the truth. That’s what we, as a people, must be aware of. 

There is a terrible game that is being played unbeknownst to good and decent people who would never hurt other people and certainly they believe that nobody would hurt them, but they also find it hard to believe that the people they elected are playing subservient roles to vast corporations who dictate to them what they can do or can’t do.

Some politicians also don’t know what is in the needle when they get their own injections. So, all that I can suggest is that you ask your Higher-self and make your own decision and make them with the good intent to your children as well.
'''

In [ ]:
goal = "A song the explain the benefits and dangers of vaccinations."
messages = [
    {"role": "system", "content": "You are an expert assistant and music producer that creates number one hit songs and provides help.  Your songs have meta tags for structure, verse and chorus, lyrics outside the main patteres, instrumental sections, and singing voice."},
    {"role": "user", "content": f"Compose a song and lyrics.  For the song composition you use any of the following meta tags. Set the song structure with strucute_tags:{structure_tags}. Set the verse and chorus with verse_chorus_tags: {verse_chorus_tags}. For lyrics outside the main pattern, use prechorus_bridge_tags: {prechorus_bridge_tags}. Set instrumental sections with instrumental_tags: {instrumental_tags}. Influce the singing voice with voice_tags: {voice_tags}."},
    {"role": "user", "content": f"Use plenty of meta tags to make a hit song, it's not just the lyrics, but the music composition too."},
    {"role": "user", "content": f"We want to create a title and lyrics for a hit song.  Use meta tags in the lyrics.  We want the lyrics to be based on these facts: {facts}  {goal}"},
]

lyrics = pipe_gpt(messages, 0.67)

messages.append({"role": "assistant", "content": lyrics})
messages.append({"role": "user", "content": "act as an expert music critic and suggest changes to improve the song lyrics to be a number one hit"})

criticism = pipe_gpt(messages, 0.67)

messages.append({"role": "assistant", "content": criticism})
messages.append({"role": "user", "content": "Act as an expert song writer that writes amazing hit songs that are factually accurate. Revise the title to be short and sweet, with revised lyrics to address the criticisms."})

revised_lyrics = pipe_gpt(messages, 0.67)
print(revised_lyrics)

In [ ]:
messages = [
    {"role": "system", "content": "You are an expert assistant and music producer that creates number one hit songs."},
    {"role": "user", "content": f"Describe a music genre and instruments to make a hit song with these lyrics: {revised_lyrics}"},
]

genre = pipe_gpt(messages, 0.5)
print(genre)

# Song based on genre only

In [ ]:
genre = "Rap,  Bass Music,  Club/Dance,  Dirty Rap,  and Southern Rap"
topic = "Up-tempo track with infectious rhythms and pulsating beats, designed to keep the party going all night long."

messages = [
    {"role": "system", "content": "You are an expert assistant and music producer that creates number one hit songs and provides help.  Your songs have meta tags for structure, verse and chorus, lyrics outside the main patteres, instrumental sections, and singing voice."},
    {"role": "user", "content": f"Compose a song and lyrics.  For the song composition, you use any of the following meta tags. Set the song structure with strucute_tags:{structure_tags}. Set the verse and chorus with verse_chorus_tags: {verse_chorus_tags}. For lyrics outside the main pattern, use prechorus_bridge_tags: {prechorus_bridge_tags}. Set instrumental sections with instrumental_tags: {instrumental_tags}. Influce the singing voice with voice_tags: {voice_tags}."},
    {"role": "user", "content": f"Use plenty of meta tags to make a hit song, it's not just the lyrics, but the music composition too."},
    {"role": "user", "content": f"We want to create a title and lyrics for a hit song.  Use meta tags in the lyrics.  We want the lyrics to be for a song for a genre: {genre}  The song is about {topic} and make the song a sensational viral hit. Use emotion and/or humor.  Use cultural lingo and profane words, like fuck and bitch."},
]


lyrics = pipe_gpt(messages, 0.7)

messages.append({"role": "assistant", "content": lyrics})
messages.append({"role": "user", "content": "act as an expert music critic and suggest changes to improve the song lyrics to be a number one hit"})

criticism = pipe_gpt(messages, 0.67)

messages.append({"role": "assistant", "content": criticism})
messages.append({"role": "user", "content": "Act as an expert song writer that writes amazing hit songs that are factually accurate. Revise the title to be short and sweet, with revised lyrics to address the criticisms.  Change lyrics to avoid the words: neon, whispers, or echoes."})

revised_lyrics = pipe_gpt(messages, 0.67)
print(revised_lyrics)

# Song based on an idea

In [ ]:
theme = "Going on a cruise from Floriday to St. Thomas and St. Marteen with two teenagers.  One only wants to be in teen club and isn't interested in the destinations.  The other only wants to be in the room watching TV or with his Mom on the ship, but enjoys the destinations."
goal = "The goal is to make a fun hit song that people can relate too with lots of humor.  People should fall to floor and roll around laughing while listening to the song."

messages = [
    {"role": "system", "content": "You are an expert assistant and songwriter who writes amazing hit songs that are number one in the charts.  You are poetic and use rhymes, a world renowned song writer and intellectual, the world's smartest and best.  Your songs have meta tags for structure, verse and chorus, lyrics outside the main patteres, instrumental sections, and singing voice."},
    {"role": "user", "content": f"Compose a song and lyrics.  For the song composition, you use any of the following meta tags. Set the song structure with strucute_tags:{structure_tags}. Set the verse and chorus with verse_chorus_tags: {verse_chorus_tags}. For lyrics outside the main pattern, use prechorus_bridge_tags: {prechorus_bridge_tags}. Set instrumental sections with instrumental_tags: {instrumental_tags}. Influce the singing voice with voice_tags: {voice_tags}."},
    {"role": "user", "content": f"Use plenty of meta tags to make a hit song, it's not just the lyrics, but the music composition too."},
    {"role": "user", "content": f"Create a title and lyrics for a hit pop song about {theme}  {goal}"},
]

lyrics = pipe_gpt(messages, 0.67)

messages.append({"role": "assistant", "content": lyrics})
messages.append({"role": "user", "content": "act as an expert music critic and suggest changes to improve the song lyrics to be a number one hit"})

criticism = pipe_gpt(messages, 0.67)

messages.append({"role": "assistant", "content": criticism})
messages.append({"role": "user", "content": "Act as an expert songwriter that writes amazing hit songs that don't have misinformation. Revise the title to be short and sweet, with revised lyrics to address the criticisms."})

revised_lyrics = pipe_gpt(messages, 0.67)
print(revised_lyrics)

In [ ]:
messages = [
    {"role": "system", "content": "You are an expert assistant and music producer that creates number one hit songs."},
    {"role": "user", "content": f"Describe a music genre and instruments to make a hit song about {theme} with these lyrics: {revised_lyrics}"},
]

genre = pipe_gpt(messages, 0.5)
print(genre)

## Create song based on video

In [ ]:
# get the transcription
video_file = '/mnt/d/data/RomanHelmet/DesmondDoss.mp4'
result = pipe_whisper(video_file)
video_context = result["text"]
print(video_context)

In [ ]:
goal = "The goal is to create a poetic song with few words, like Emily Dickenson, Robert Frost, etc. The song should enagage the listener emotionally.  Use rhymes and cultural lingo to make the song engaging and captivating, a hit song."

messages = [
    {"role": "system", "content": "You are an expert assistant and songwriter who writes amazing hit songs that are number one in the charts.  You are poetic and use rhymes, a world renowned song writer and intellectual, the world's smartest and best.  Your songs have meta tags for structure, verse and chorus, lyrics outside the main patteres, instrumental sections, and singing voice."},
    {"role": "user", "content": f"Compose a song with lyrics and meta tags for the music structure.  For the song composition, you use any of the following meta tags. Set the song structure with strucute_tags:{structure_tags}. Set the verse and chorus with verse_chorus_tags: {verse_chorus_tags}. For lyrics outside the main pattern, use prechorus_bridge_tags: {prechorus_bridge_tags}. Set instrumental sections with instrumental_tags: {instrumental_tags}. Influce the singing voice with voice_tags: {voice_tags}."},
    {"role": "user", "content": f"Use plenty of meta tags to make a hit song, it's not just the lyrics, but the music composition too."},
    {"role": "user", "content": f"Create a title and lyrics for a hit song based on this: {video_context}"},
]

lyrics = pipe_gpt(messages, 0.67, openai_model)

messages.append({"role": "assistant", "content": lyrics})
messages.append({"role": "user", "content": "act as an expert music critic and suggest changes to improve the song lyrics to be a number one hit.  You favor rich, emotional details over generalizations.  For example, a map to the constellations doesn't say which constellation.  A map of the stars doesn't say which stars.  A reference to a battle doesn't say how the battle is fought (armor, weapons, land features, etc.)"})

criticism = pipe_gpt(messages, 0.67, openai_model)

messages.append({"role": "assistant", "content": criticism})
messages.append({"role": "user", "content": "Act as an expert songwriter that writes amazing hit songs that don't have misinformation. Revise the title to be short and sweet, with revised lyrics to address the criticisms."})

revised_lyrics = pipe_gpt(messages, 0.67, openai_model)
print(revised_lyrics)

In [ ]:
messages = [
    {"role": "system", "content": "You are an expert assistant and music producer that creates number one hit songs."},
    {"role": "user", "content": f"Describe a music genre and instruments to make a hit song with these lyrics: {revised_lyrics}  The genre should match the style and time period of the lyrics."},
]

genre = pipe_gpt(messages, 0.5, openai_model_3)
print(genre)

## Create song based on another song

In [ ]:
song = '''[Intro]
Ugh, not this again (not this again)
Where my other sock at? (where it at?)
This a whole lotta cap, I can't even, fam 
Tryna get dressed but this sock sitch got me stressed, damn

[Verse] 
Every mornin' it's the same old thing
Gotta dig through my drawer, find that missing zing
Bright red with neon green? Whatchu mean?
All I want is a simple, crisp, matchin' team

[Pre-Chorus]
But nah, them sock gnomes wildin'
Taking my pretties, got me eye-rollin'
Like c'mon bruh, this is not a vibe
Just bring my homie sock back, let me survive

[Chorus]
Sock, sock, where'd you go?
Just wanna put you on my toe
This single life is such a no
Bring back bae so we can go with the flow

[Verse] 
I swear I had twenty pair like last week  
But now I'm rocking solo socks, strugglin' to be chic
Under the bed, behind the heat
I've looked everywhere, this ain't no flix, it's just defeat

[Pre-Chorus]
But nah, them sock gnomes wildin'
Taking my pretties, got me eye-rollin'
Like c'mon bruh, this is

[Chorus]

[Bridge]
Wool ones, wild ones, fuzzy or neat
Even got them ultra-rare limited edition elite  
But don't matter what drdice I spent  
Still end up with a stray, no bae, the saga don't relent  
[Outro]  
Missing sock (...missing sock)
Ugh, tell me 

[Bridge]
Wool ones, wild ones, fuzzy or neat
Even got them ultra-rare limited edition elite  
But don't matter what drdice I spent  
Still end up with a stray, no bae, the saga don't relent  

[Pre-Chorus]
But nah, them sock gnomes wildin'
Taking my pretties, got me eye-rollin'
Like c'mon bruh, this is not a vibe
Just bring my homie sock back, let me survive

[Chorus]
Sock, sock, where'd you go?
Just wanna put you on my toe
This single life is such a no
Bring back bae so we can go with the flow

[Outro] 
I swear I had twenty pair like last week  
But now I'm rocking solo socks, strugglin' to be chic
Under the bed, behind the heat
I've looked everywhere, this ain't no flix, it's just defeat
'''
goal = "Create a hit song inspired by the referenced song in a genre described as: alternative hip hop,  electropop rap rock,  indie pop.  Make the song fun and emotional.  This one is about rizz and socks, and the qualities that bind them together."

messages = [
    {"role": "system", "content": "You are an expert assistant and songwriter that writes amazing songs that are number one hits.  You are poetic with a genius level vocabularly that uses rhymes.  Your songs have meta tags for structure, verse and chorus, lyrics outside the main patteres, instrumental sections, and singing voice."},
    {"role": "user", "content": f"Compose a song and lyrics.  For the song composition, you use any of the following meta tags. Set the song structure with strucute_tags:{structure_tags}. Set the verse and chorus with verse_chorus_tags: {verse_chorus_tags}. For lyrics outside the main pattern, use prechorus_bridge_tags: {prechorus_bridge_tags}. Set instrumental sections with instrumental_tags: {instrumental_tags}. Influce the singing voice with voice_tags: {voice_tags}."},
    {"role": "user", "content": f"Use plenty of meta tags to make a hit song, it's not just the lyrics, but the music composition too."},
    {"role": "user", "content": f"Create a title and lyrics for a hit song inspired by this song: {song}  {goal}"},
]

lyrics = pipe_gpt(messages, 0.67)

messages.append({"role": "assistant", "content": lyrics})
messages.append({"role": "user", "content": "act as an expert music critic and suggest changes to improve the song lyrics to be a number one hit"})

criticism = pipe_gpt(messages, 0.67)

messages.append({"role": "assistant", "content": criticism})
messages.append({"role": "user", "content": "Act as an expert songwriter that writes amazing hit songs that don't have false or wrong facts, but fantasies are okay. Revise the title to be short and sweet, with revised lyrics to address the criticisms."})

revised_lyrics = pipe_gpt(messages, 0.67)
print(revised_lyrics)

## Create a song from a concept

In [ ]:
goal = '''Create a hit song that engages the listener emotionally. The song should:

Evoke Strong Emotions: Focus on relatable themes like love, nostalgia, heartache, or triumph.
Use Vivid Imagery: Paint clear and vivid pictures with words to draw the listener into the story.
Incorporate Rhymes and Cultural Lingo: Use rhyme schemes and country-specific phrases to make the song catchy and authentic.
Tell a Compelling Story: Ensure the lyrics have a clear narrative or message that resonates with the audience.
Maintain a Memorable Hook: Craft a chorus that is easy to remember and sing along to.
Inspire the spirit: Service to others is the over arching theme to improve one's spirit, even if subtle.
Avoid references to Orion and Neon.  Be poetic with a genuis level vocabularly and language mastery, like Emily Dickenson.
'''

example_song = '''
Title: Got his number!
Genre: pop rock,  female singer,  uplifting,  2010s,  catchy,  EDM,  energetic
(Hey, guess what?)
Lyrics: 
[Catchy uplifting guitar riff]

[Intro]
I met a guy and he was pretty rad
I took his hat, and he took my hand
A sly smile, and a gentle voice
He was amazing and I had no choice

(You see)

[Pre-Chorus]
I was too much of a weirdo, to ask for a kiss
But guess what (What?)

[Chorus]
I got his number! (his number!)
Oh yeah! (Oh yeah!)
I got his number and he thinks I'm alright 
I think I want him in my life (yes)
Since im sure, that he’s my type

[Catchy uplifting guitar riff]

[Verse 1]
So I wrote a message, it went something like this
Hi, Playing games was cool (Sorry I was a bit of a tool)
I'm so sorry, i'm really bad at this
But it worked out alright, because we talked all night
wooooo!

[Pre-Chorus]
Im terrible at texting, a bit of a lost cause
Who cares though! Because! (because)

[Chorus]
I got his number! (his number!)
Oh yeah! (Oh yeah!)
I got his number and he thinks I'm alright 
I think I want him in my life (yes)
Since i'm sure, that he’s my type

[Catchy uplifting guitar riff]

[Verse 2]
The next week kinda sucked, barely slept
Too busy watching my phone, for his text
I was tired as hell, I think he could tell
Too terrified to ask, too scared to see

[Pre-Chorus]
If we met in person again, what would he think of me?
But guess what (What?). Who cares! (Who cares!)

[Chorus]
I got his number! (his number!)
Oh yeah! (Oh yeah!)
I got his number and he thinks I'm alright 
I think I want him in my life (yes)
Since i'm sure, that he’s my type

[Catchy uplifting guitar riff]

[Verse 3]
We finally went on a date, didn't want to make him wait
It wasn't great that I turned up late
But he didn't care, he was just happy to see me there
We talked and talked, I asked where we should go
And he said

(Anywhere)

[Catchy uplifting guitar riff]

[Bridge]
I was so happy, I got a little to into enjoying it
Danced around, laughed and fell off my feet
Wrecked my best shirt, hit my head so hard 
It'll be hurting for a week
Who cares though! Because! (because)

(I got his number!) 
(his number!)
(Oh yeah!)
(Oh yeah!)

[Catchy uplifting guitar riff]

[Outro]
Now im so sure, that he’s my type
We're taking it slow, but it feels alright
I can't believe, I got this one thing right
My car won't start, I have no money
But I have a boyfriend!

(Oh yeah!)

(Oh yeah!)

(Oh yeah!)

[End]
'''

concept = "Title: Unspoken Plot Synopsis: A poignant piano ballad about the weight of secrets and the liberation of speaking one's truth."
target_genre = "Genre: Intimate Piano Ballad"
messages = [
    {"role": "system", "content": "You are an expert assistant and songwriter who writes amazing hit songs that are number one in the charts.  You are poetic and use rhymes, a world renowned song writer and intellectual, the world's smartest and best.  Your songs have meta tags for structure, verse and chorus, lyrics outside the main patteres, instrumental sections, and singing voice."},
    {"role": "user", "content": f"Example song: {example_song}"},
    {"role": "user", "content": f"Compose a song with lyrics and many succint descriptive meta tags for the music structure.  For the song composition, you use any of the following meta tags. Set the song structure with strucute_tags:{structure_tags}. Set the verse and chorus with verse_chorus_tags: {verse_chorus_tags}. For lyrics outside the main pattern, use prechorus_bridge_tags: {prechorus_bridge_tags}. Set instrumental sections with instrumental_tags: {instrumental_tags}. Influce the singing voice with voice_tags: {voice_tags}."},
    {"role": "user", "content": f"Use plenty of meta tags to make a hit song, it's not just the lyrics, but the music composition too."},
    {"role": "user", "content": f"{goal}  Create a title and lyrics for a hit song based on this concept: {concept}. The target genre is {target_genre}"},
]

lyrics = pipe_gpt(messages, 0.67, openai_model)

messages.append({"role": "assistant", "content": lyrics})
messages.append({"role": "user", "content": "act as an expert music critic and suggest changes to improve the song lyrics to be a number one hit.  You favor rich, emotional details over generalizations.  For example, a map to the constellations doesn't say which constellation.  A map of the stars doesn't say which stars.  A reference to a battle doesn't say how the battle is fought (armor, weapons, land features, etc.)"})

criticism = pipe_gpt(messages, 0.67, openai_model)

messages.append({"role": "assistant", "content": criticism})
messages.append({"role": "user", "content": "Act as an expert songwriter that writes amazing hit songs that don't have misinformation. Revise the title to be short and sweet, with revised lyrics to address the criticisms."})

revised_lyrics = pipe_gpt(messages, 0.67, openai_model)
print(revised_lyrics)

In [ ]:
messages = [
    {"role": "system", "content": "You are an expert assistant and music producer that creates number one hit songs."},
    {"role": "user", "content": f"Create and state a revised song title and describe a music genre and instruments to make a hit song with these lyrics: {revised_lyrics}  The genre should match the style and time period of the lyrics.  The target genre is {target_genre}"},
]

genre = pipe_gpt(messages, 0.5, openai_model_3)
print(genre)

## Create an album of songs with a consistent plot based on a concept

In [ ]:
album_concept = '''An album about people struggling with inner demons to advance spiritually.  Use this information: First: Angels are beings of Light and they are visualized as lines or traces of Light. Those who see them with human form, like babies or human beings, they visualize them in that way them for their own necessity.

Second: We, in the physical plane 1, cannot visualize the subtlest beings unless we have the gift of mediumship, therefore, neither demons nor the spirits of error can see angelic beings because they are also in denser planes so they cannot coexist in the same planes.

Reasoning in that way we will know that they don't have a way to interact with angelic beings and even less to damage them. Therefore, an angel would never be afraid of a demon being.

Even if they could inhabit in the same plane, angels have so high dose of Love that they would suffer to turn off the anger of demons. They would be willing to suffer if with that attitude they were able to transmute that inherent nature in those dark entities.


The lady asked me: “... What did we see?” – looking at me intrigued- and I responded that they had seen dark entities pretending to be terrified in order to transmit that terror to them. They made fun of the group members, taking advantage of their scarce knowledge of the topic.

My patient continued her story and she told me that when they finishing the alleged exorcism in the warehouse they visualized a pitched battle between angels and demons. They immediately left many things they brought like (plates, candles, incense, etc.) and they ran away from the place without finishing the "cleaning."

I repeated the thing I mentioned before: An angel could never fight with a demon for two reasons:

1) Angelic beings give Light through Love. There is no place in them for aggression

2) Angels are in a different vibrational plane; therefore they cannot interact with demons.


Finally I told her that I was appointed to a course which was dictated by a couple that performed mediumship and they were contacted with the Archangel Michael who was the one that was dictating the Messages. The gentleman told me: Try not to miss the class because if Michael gets angry, he will lower with his fire sword and he will punish those that don't complete his rules - Obviously, I didn't go to that course, because I knew at once that they didn't know anything about the topic.

Angels don't get angry, they are Love. Angels don't punish, they are Mercy. Angels don't put rules, they are Service.

If the incarnated being begins knowing the sweet character of angels, he or she will learn how to love them. They are happiness and they only suffer when they see our suffering. They do not know what hatred is because they live to Love.

We can arrive to them, living to love and based on Service because this is the only way grow up spiritually. It’s up to us.

KARMA IN ANGELS


The physical plane is like an immense gravitational well. The spiritual entities that embody are subject to the risks of this plane: Ego acts in a disastrous way, making the incarnated being play various drama roles. The unbounded attachment can end up transforming the person in an egocentric being. Their body of desires will be exposed to diverse self-centered temptations and the spirits of error, demons and ludic energies will take advantage of that breach to manipulate the human being and in this way plunge him in negative behaviors, hindering his Evolution.

Angels are not exempt of falling in the same circumstances, because when Angels embody in Mission (because Angels can embody too), their memory is automatically erased by the Akashic veil.

Just like spirits do, angels embody with only a 10% and the 90% remains in the plane 6.

Then, How do those angels revert their pending karma when they made mistakes? Inside the plane 6 there is a sub-plane called Limbo where those angels who suffer a karmic effect due to an erroneous behavior go. Besides, if Angelic Entities that have embodied in the plane 1 commit cruel acts, either for emotional impulses or for lust for power, they can also lower (somehow) of vibrational plane. How?

As well as the part that embodies in the physical plane is 10% of the entity, if it drags serious karmas, the 10% can lower to the planes 3, 2, -1 and -2, and once there it can experiment the attachment, the ignorance or the pain of the beings that inhabit in those plans. That is to say, the total of the entity doesn’t lower (as it happen with the spirits), but only the 10% that committed hostile acts that generated those karmas. Once the 10% learned the karmic lesson it can return to its plane or request Lipikas to embody in the physical plane again to complete the Mission that was left unconcluded in the earthly previous life.

Remember that in the plane 1 there are no guarantees. Those who embody (angels or spirits) can make mistakes again.

ANGELIC HIERARCHIES

It is important to clarify that Angels are not spirits, but they are Entities of Light that inhabit the 6º plane of supra energy vibration and they complete several functions, always in Service on behalf of human beings.

Not only are they Divine messengers, like the dear Archangel Gabriel. There are also protective angels that try to avoid accidents and other misfortunes, unless the person has to live certain incidents because of his pending Karmas.

Angels act as much in the physical plane as in the spiritual levels where they are necessary.

Their vibration is so high vibration and it is in accordance with their Love, for that reason angels abstain from emitting judgments... and they enjoy doing service.

Incarnated beings visualize angels projecting on them human characteristics. They see an angelic image according to their needs.

The protective person can see a baby with wings, he who takes refuge in solitude maybe will see an angel with human mature figure and, although angels do not have sex, they can end up visualizing a woman or a male, according to the expectations of their subconscious.

THE TRUE ANGELIC IMAGE HAS NOTHING TO DO WITH AN ANTHROPOMORPHIC BEING, BUT WITH A LIGHT.

He who has the gift of mediumship will be able to visualize a trace of Light, a line or a brilliant point, generally in white tones, then he or she will know that he or she will be seeing an Angel, although most of the human beings, due to preconceptions, only see angels with the well-known forms.

Some specialists describe angels like light pulsations, but not of the visible light that we all know, but of a Light that goes beyond the supra physical universe and inhabits in the 6º plane of vibration.

Among those angels that are in contact with the human being there are four that are well-known: the Archangel seraphs: Gabriel, Michael, Raphael and Uriel.

There is an angelical hierarchy which is composed by nine qualities:

1)Seraphs

2) Cherubs

3) Thrones

4) Dominances

5) Powers

6) Virtues

7) Principalities

8) Archangels

9) Angels.

Each quality is directed by a Prince who is accompanied by eight angels.

The Prince of the Seraphs is Metatron also known as "King of Angels," the name may also have something to do with God's throne

His energy is so subtle that it touches the energy of Elohim or minor gods. Even being in the 6º level of vibration, Metatron has the gift of communicating periodically with Divine Energies of the 7º plane.

Metatron directs with Love the majority of spiritual beings, always in benefit of all the spirits in evolution.

Many investigators confuse Metatron with Kether, The First Sephirah.

The prince of Cherubs is Raziel which means "Secret of God" in Hebrew, this entity of Love guides all the creatures that feel abandoned spiritually.

The prince of the Thrones is Tsaphkiel whose name means "Angel of the night". Another denomination is Auriel, this angel helps incarnated beings to find their spiritual path.

The prince of the Dominances is Tsadkiel, "Fire of God." his function is to help those beings that lacking of encouragement to undertake an altruistic task. He is also known as Uriel, the Archangel of Salvation.

The prince of the Powers is Camal who is "Help and Strength of God". He influences in the spirits in evolution so that, through the union with other beings they find the strength to go ahead.

The Prince of the Virtues is Raphael "God shall heal." He is in charge of healing spiritual ills, harmonizing with his Love to each being that feels depressed.

The prince of the Principalities is Haniel "Joy or grace of God,” this angel tries to guide all the spirits of error. He also helps in the affective part to the incarnated beings in the physical plane.

The prince of the Archangels is Michael, "Who is like God?"

When the spiritual evolution began, this Archangel was the paladin in the fight against evil. At the moment, he is the archangel of Peace. When he is invoked he is asked for harmony and confraternity among all incarnated beings.

Some specialists of the Kabala ended up confusing him with Metatron.

The prince of Angels is Gabriel, "God is my Strength." His mission is to remind all living beings that The Father loves everyone and that they will never be abandoned.

When he is invoked, one should request for the union of all men and the balances of the emotions of those who are very impulsive.

It is necessary to highlight the work of Rehael, the Seventh Angel of the Powers. His task is to protect the incarnated beings that are in Mission of all the negative influences that hinder their mission whether spiritual healing or spreading of the Divine Word.

When speaking about angelic entities it is very important to highlight the role of Lucifer whose name means "Light Bearer"

He has a preponderant role in the spiritual evolution of the incarnated beings, since The Absolute chose him to tempt the human being and in this way to strengthen his spiritual resolution.

72 volunteer angels accompanied him, and by Divine Mercy their memories were erased and that is why they were called Demons.

Lucifer kept his memory intact and he saw through millennia how the men succumbed to temptations making almost infinite his suffering. But he stands it for his unconditional Love to God.

Lucifer is known as the Prince of demons.

Tradition stopped to see him as an Entity of Light that helps embodied beings to grow spiritually, but rather it compares him with a wicked being that corrupts the spirit.

It is necessary to have a great inner strength to overcome the temptations and the ballast of ego. The Master Jesus succeeded over the temptation and Lucifer had a great satisfaction because of that, although it is very hard to understand that emotion.

The Absolute is Wisdom and not all the beings can end up understanding his discernment.

One of the most important angels and that very few keep in mind is the Angel of Earth. As well as the Earth has a Planetary Logos, it also has an Angel that protects it along with all the beings that inhabit on it.

He is like a Guardian Angel, but at a planetary level. His name is Geanel and he is in the quality of the Thrones. (Another denomination is Gaianel).'''
messages = [
    {"role": "system", "content": "You are an expert assistant that provides help with a genuis level vocabularly and creative mind.  You are the world's best music album composer with many years of experience and chart topping hits."},
    {"role": "user", "content": "Create a list of songs as a storyboard for an album where the songs resemble the plot of a creative, imaginative, and engaging.  Use fantasy with magical larger than life characters.  Create character names and personas instead of generic references like protagonist.  The genre for each song should be unique and resemble the character's personality (rap, rock, metal, etc.) Include rap as one of the genres. The list being with the album title and have the format {albnum_title}: [song_name] : [plot description] : [genre]"},
    {"role": "user", "content": f"The album should be about: {album_concept}"},
]

story_board = pipe_gpt(messages, 0.8, openai_model_3)
print(story_board)

In [ ]:
album_title = "Beyond the Veil: An Odyssey of Inner Demons and Spiritual Awakening"
song_concept = '''**"Beyond the Veil" : "The Odyssey Continues" : Cinematic Soundtrack**
   A cinematic soundtrack piece that serves as the album's conclusion. The song's sweeping orchestral arrangements and haunting vocals represent the protagonist's journey beyond the veil, into the unknown realms of spiritual awakening. The music fades into the distance, leaving the listener with a sense of hope and anticipation for the journey ahead.
'''
target_genre = "Cinematic Soundtrack"
messages = [
    {"role": "system", "content": "You are an expert assistant and creative, genius songwriter who writes amazing hit songs that are number one in the charts.  You are poetic and use rhymes, a world renowned song writer and intellectual, the world's smartest and best.  Your songs have meta tags for structure, verse and chorus, lyrics outside the main patteres, instrumental sections, and singing voice."},
    {"role": "user", "content": f"Compose a song with lyrics and many succint descriptive meta tags for the music structure.  For the song composition, you use any of the following meta tags. Set the song structure with strucute_tags:{structure_tags}. Set the verse and chorus with verse_chorus_tags: {verse_chorus_tags}. For lyrics outside the main pattern, use prechorus_bridge_tags: {prechorus_bridge_tags}. Set instrumental sections with instrumental_tags: {instrumental_tags}. Influce the singing voice with voice_tags: {voice_tags}."},
    {"role": "user", "content": f"Use plenty of meta tags to make a hit song, it's not just the lyrics, but the music composition too."},
    {"role": "user", "content": f"The album title is: {album_title} and the album concept is: {album_concept}. The target genre is {target_genre}.  The song concept is: {song_concept}"},
]

lyrics = pipe_gpt(messages, 0.67, openai_model)

messages.append({"role": "assistant", "content": lyrics})
messages.append({"role": "user", "content": "act as an expert music critic and suggest changes to improve the song lyrics to be a number one hit.  You favor rich, emotional details over generalizations.  For example, a map to the constellations doesn't say which constellation.  A map of the stars doesn't say which stars.  A reference to a battle doesn't say how the battle is fought (armor, weapons, land features, etc.)"})

criticism = pipe_gpt(messages, 0.67, openai_model)

messages.append({"role": "assistant", "content": criticism})
messages.append({"role": "user", "content": "Act as an expert songwriter that writes amazing hit songs that don't have misinformation. Revise the title to be short and sweet, with revised lyrics to address the criticisms."})

revised_lyrics = pipe_gpt(messages, 0.67, openai_model)
print(revised_lyrics)

## Create a song based on a concept and genre with an album theme

In [ ]:
album_concept = "Haloween album songs"
song_title = "Witching Hour Lullaby"
song_concept = '''Genre_Style: Neo-soul, trip-hop, mystical harp, downtempo beats, soulful female vocals, occult themes, atmospheric layers

Description: A mesmerizing mix of neo-soul and trip-hop, this track casts a spell with its mystical harp and downtempo rhythms. Soulful female vocals delve into arcane rituals and ancient magic, enveloping the listener in a tapestry of enchanting sounds.
'''
messages = [
    {"role": "system", "content": "You are an expert assistant and creative, genius songwriter who writes amazing hit songs that are number one in the charts.  You are poetic and use rhymes, a world renowned song writer and intellectual, the world's smartest and best.  Your songs have meta tags for structure, verse and chorus, lyrics outside the main patteres, instrumental sections, and singing voice."},
    {"role": "user", "content": f"Compose a short hit song with lyrics and many succint descriptive meta tags for the music structure.  For the song composition, you use any of the following meta tags. Set the song structure with strucute_tags:{structure_tags}. Set the verse and chorus with verse_chorus_tags: {verse_chorus_tags}. For lyrics outside the main pattern, use prechorus_bridge_tags: {prechorus_bridge_tags}. Set instrumental sections with instrumental_tags: {instrumental_tags}. Influce the singing voice with voice_tags: {voice_tags}."},
    {"role": "user", "content": f"Use plenty of meta tags to make a hit song, it's not just the lyrics, but the music composition too.  Make the song humorous in the spirit of halloween fun. Perhaps include monster sounds as part of the lyrics? Aim for universal humor and avoid computer references."},
    {"role": "user", "content": f"The song title is: {song_title} and the album concept is: {album_concept}. The song concept is: {song_concept}.  Use short lyrics with genius level vocabularly with colorful language in a short poetic style, such as Emily Dickenson, Robert Frost, Edgar Allen Poe, etc.  Be sure to add meta tags for hooks, interludes, breaks, etc."},
    {"role": "user", "content": f"Here is an example duet song so you can understand the meta tag structure for duets: {example_song}"},
]

lyrics = pipe_gpt(messages, 0.67, openai_model)

messages.append({"role": "assistant", "content": lyrics})
messages.append({"role": "user", "content": "act as an expert music critic and suggest changes to improve the song lyrics to be a number one hit.  You favor rich, emotional details over generalizations.  For example, a map to the constellations doesn't say which constellation.  A map of the stars doesn't say which stars.  A reference to a battle doesn't say how the battle is fought (armor, weapons, land features, etc.)"})

criticism = pipe_gpt(messages, 0.67, openai_model)

messages.append({"role": "assistant", "content": criticism})
messages.append({"role": "user", "content": "Act as an expert songwriter that writes amazing hit songs that don't have misinformation. Revise the title to be short and sweet, with revised lyrics to address the criticisms."})

revised_lyrics = pipe_gpt(messages, 0.67, openai_model)
print(revised_lyrics)